# heat1d + JPL Horizons: Real Ephemeris Illumination

This notebook demonstrates the `heat1d.horizons` module, which queries the
[JPL Horizons](https://ssd.jpl.nasa.gov/horizons/) ephemeris system for
real solar geometry and uses it to drive the thermal model.

**Requirements:** Network access (for Horizons API queries).

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import planets

import heat1d
from heat1d.horizons import (
    query_horizons,
    horizons_to_flux,
    fetch_solar_flux,
    get_body_id,
    HORIZONS_BODY_IDS,
)
from heat1d.plotting import diurnal_curves, profile_plot

%matplotlib inline

## 1. Query Horizons for Solar Position

The `query_horizons` function queries the Sun's apparent position
(azimuth, elevation, distance) as seen from a surface point on any body.
Here we query from the lunar equator (lon=0, lat=0) over one full
lunar day (~29.5 Earth days).

In [ ]:
result = query_horizons(
    body_id="301",       # Moon
    lon_deg=0.0,
    lat_deg=0.0,
    start_time="2024-06-01 00:00",
    stop_time="2024-07-01 00:00",
    step_size="3h",
)

print(f"Samples:  {result['n_samples']}")
print(f"dt:       {result['dt_seconds']/3600:.1f} hours")
print(f"Elev range: {result['solar_elevation_deg'].min():.1f} to "
      f"{result['solar_elevation_deg'].max():.1f} deg")
print(f"Distance:   {result['observer_range_au'].min():.4f} to "
      f"{result['observer_range_au'].max():.4f} AU")

In [ ]:
# Convert timestamps to fractional Earth days since start
t_days = np.arange(result['n_samples']) * result['dt_seconds'] / 86400.0

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

# Solar elevation
elev = result['solar_elevation_deg']
ax1.plot(t_days, elev, 'k-', lw=1)
ax1.axhline(0, color='orange', ls='--', lw=0.8, label='Horizon')
ax1.fill_between(t_days, elev, 0, where=(elev > 0), alpha=0.3, color='gold', label='Daytime')
ax1.fill_between(t_days, elev, 0, where=(elev <= 0), alpha=0.15, color='navy', label='Night')
ax1.set_ylabel('Solar elevation [deg]')
ax1.legend(loc='upper right', fontsize=9)
ax1.set_title('Sun from Moon equator (lon=0, lat=0) -- JPL Horizons')

# Sun-observer distance
ax2.plot(t_days, result['observer_range_au'], 'r-', lw=1)
ax2.set_ylabel('Sun distance [AU]')
ax2.set_xlabel('Earth days since 2024-06-01')

fig.tight_layout()
plt.show()

## 2. Convert to Absorbed Surface Flux

`horizons_to_flux` applies the angle-dependent albedo model
(Keihm 1984, Hayne et al. 2017 Eq. A8) to compute the absorbed
solar flux at each timestep.

In [ ]:
flux = horizons_to_flux(
    result['solar_elevation_deg'],
    result['observer_range_au'],
    planets.Moon,
)

fig, ax = plt.subplots(figsize=(10, 3.5))
ax.plot(t_days, flux, 'k-', lw=1)
ax.fill_between(t_days, flux, alpha=0.3, color='gold')
ax.set_xlabel('Earth days since 2024-06-01')
ax.set_ylabel('Absorbed flux [W/m$^2$]')
ax.set_title('Absorbed solar flux at Moon equator (Horizons ephemeris)')
ax.set_xlim(t_days[0], t_days[-1])
fig.tight_layout()
plt.show()

print(f"Peak flux: {flux.max():.1f} W/m^2")
print(f"Day fraction: {np.count_nonzero(flux > 0) / len(flux):.1%}")

## 3. Run the Thermal Model with Horizons Flux

The `fetch_solar_flux` convenience function queries Horizons and returns
a `(flux_array, dt)` pair that plugs directly into the `Model` constructor.
The model uses its built-in Fourier solver for equilibration, then
switches to the Horizons-derived flux for the output phase.

In [ ]:
# Fetch flux from Horizons (one lunar day, ~0.5 hr output resolution)
output_interval_s = planets.Moon.day / 48  # 48 steps per lunar day

flux_hz, dt_hz, meta = fetch_solar_flux(
    planet_name="Moon",
    lon_deg=0.0,
    lat_deg=0.0,
    start_time="2024-06-15 00:00",
    stop_time="2024-07-14 18:00",
    output_interval_s=output_interval_s,
)

print(f"Horizons body: {meta['body_id']}")
print(f"Samples: {meta['n_samples']}, dt = {dt_hz:.0f} s ({dt_hz/3600:.1f} hr)")

# Run thermal model
config = heat1d.Configurator(
    solver="implicit",
    output_interval=dt_hz,
)

ndays = len(flux_hz) * dt_hz / planets.Moon.day

model_hz = heat1d.Model(
    planet=planets.Moon,
    lat=0.0,
    ndays=ndays,
    config=config,
    flux_series=flux_hz,
    flux_dt=dt_hz,
)
model_hz.run()

print(f"\nT_max  = {model_hz.T[:, 0].max():.1f} K")
print(f"T_min  = {model_hz.T[:, 0].min():.1f} K")
print(f"T_mean = {model_hz.T[:, 0].mean():.1f} K")

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 7), sharex=True)

# Flux
t_hr = np.arange(len(flux_hz)) * dt_hz / 3600.0
ax1.plot(t_hr, flux_hz, 'k-', lw=0.8)
ax1.fill_between(t_hr, flux_hz, alpha=0.3, color='gold')
ax1.set_ylabel('Absorbed flux [W/m$^2$]')
ax1.set_title('Horizons-driven thermal model: Moon equator (lon=0)')

# Surface temperature
t_model_hr = model_hz.lt / 24.0 * planets.Moon.day / 3600.0  # convert to Earth hours
ax2.plot(t_model_hr, model_hz.T[:, 0], 'r-', lw=1)
ax2.set_ylabel('Surface temperature [K]')
ax2.set_xlabel('Time since UTC start [Earth hours]')

fig.tight_layout()
plt.show()

## 4. Compare Horizons vs. Analytical Orbits

Run the same simulation using the default analytical orbital model
(from `orbits.py`) and compare surface temperatures.

In [ ]:
# Analytical model (default, no external flux)
config_ref = heat1d.Configurator(
    solver="implicit",
    output_interval=output_interval_s,
)
model_ref = heat1d.Model(
    planet=planets.Moon,
    lat=0.0,
    ndays=1,
    config=config_ref,
)
model_ref.run()

print(f"Analytical: T_max={model_ref.T[:, 0].max():.1f} K, "
      f"T_min={model_ref.T[:, 0].min():.1f} K")
print(f"Horizons:   T_max={model_hz.T[:, 0].max():.1f} K, "
      f"T_min={model_hz.T[:, 0].min():.1f} K")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))

# Analytical: plot against local time (0-24 scale)
ax.plot(model_ref.lt, model_ref.T[:, 0], 'b-', lw=1.5,
        label='Analytical orbits', alpha=0.8)

# Horizons: plot against local time equivalent
# model_hz.lt gives hours since start, wrap to 0-24
lt_hz = model_hz.lt % 24.0
ax.plot(lt_hz, model_hz.T[:, 0], 'r--', lw=1.5,
        label='JPL Horizons', alpha=0.8)

ax.set_xlabel('Local time [hours]')
ax.set_ylabel('Surface temperature [K]')
ax.set_title('Moon equator: Analytical vs Horizons illumination')
ax.legend()
ax.set_xlim(0, 24)
fig.tight_layout()
plt.show()

## 5. Different Latitude: Apollo 15 Landing Site

The Apollo 15 landing site at Hadley Rille (26.1N, 3.6E) is a
mare region. Let's query Horizons for the real illumination
conditions there.

In [ ]:
import copy

# Apollo 15: Hadley Rille (mare basalt, lower albedo)
apollo15_lat = 26.1
apollo15_lon = 3.6
mare_albedo = 0.06

# Query Horizons
flux_a15, dt_a15, meta_a15 = fetch_solar_flux(
    planet_name="Moon",
    lon_deg=apollo15_lon,
    lat_deg=apollo15_lat,
    start_time="2024-06-15 00:00",
    stop_time="2024-07-14 18:00",
    output_interval_s=output_interval_s,
)

# Create mare planet variant
moon_mare = copy.copy(planets.Moon)
moon_mare.albedo = mare_albedo

# Recompute flux with mare albedo
result_a15 = query_horizons(
    body_id="301",
    lon_deg=apollo15_lon,
    lat_deg=apollo15_lat,
    start_time="2024-06-15 00:00",
    stop_time="2024-07-14 18:00",
    step_size=meta_a15['step_size'],
)
flux_a15_mare = horizons_to_flux(
    result_a15['solar_elevation_deg'],
    result_a15['observer_range_au'],
    moon_mare,
)

# Run model with mare albedo
config_a15 = heat1d.Configurator(
    solver="implicit",
    output_interval=dt_a15,
)
ndays_a15 = len(flux_a15_mare) * dt_a15 / planets.Moon.day

model_a15 = heat1d.Model(
    planet=moon_mare,
    lat=math.radians(apollo15_lat),
    ndays=ndays_a15,
    config=config_a15,
    flux_series=flux_a15_mare,
    flux_dt=dt_a15,
)
model_a15.run()

print(f"Apollo 15 site ({apollo15_lat}N, {apollo15_lon}E, A0={mare_albedo}):")
print(f"  T_max  = {model_a15.T[:, 0].max():.1f} K")
print(f"  T_min  = {model_a15.T[:, 0].min():.1f} K")
print(f"  T_mean = {model_a15.T[:, 0].mean():.1f} K")

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 7))

t_hr_a15 = np.arange(len(flux_a15_mare)) * dt_a15 / 3600.0

# Flux comparison: equator vs Apollo 15
ax1.plot(t_hr, flux_hz, 'b-', lw=0.8, alpha=0.7, label='Equator (A$_0$=0.12)')
ax1.plot(t_hr_a15, flux_a15_mare, 'r-', lw=0.8, alpha=0.7,
         label=f'Apollo 15 ({apollo15_lat}N, A$_0$={mare_albedo})')
ax1.set_ylabel('Absorbed flux [W/m$^2$]')
ax1.legend(fontsize=9)
ax1.set_title('Absorbed flux: equator vs Apollo 15 (Horizons)')

# Temperature comparison
t_model_hr_a15 = model_a15.lt / 24.0 * planets.Moon.day / 3600.0
ax2.plot(t_model_hr, model_hz.T[:, 0], 'b-', lw=1, alpha=0.7, label='Equator')
ax2.plot(t_model_hr_a15, model_a15.T[:, 0], 'r-', lw=1, alpha=0.7,
         label=f'Apollo 15 ({apollo15_lat}N)')
ax2.set_ylabel('Surface temperature [K]')
ax2.set_xlabel('Time since UTC start [Earth hours]')
ax2.legend(fontsize=9)

fig.tight_layout()
plt.show()

## 6. Supported Bodies

The module includes a mapping from `planets` package names to
Horizons body IDs. Any Horizons-supported body can be used
with the `--body-id` CLI option.

In [ ]:
print(f"{'Body':<12s} {'Horizons ID':>12s}")
print("-" * 25)
for name, bid in sorted(HORIZONS_BODY_IDS.items()):
    print(f"{name:<12s} {bid:>12s}")

## CLI Usage

The same functionality is available from the command line:

```bash
# Moon equator, one lunar day, Horizons illumination
heat1d --use-spice --lat 0 --lon 0 --start-time "2024-06-15 12:00" --ndays 1

# Apollo 15 site with mare albedo
heat1d --use-spice --lat 26.1 --lon 3.6 --albedo 0.06 \
       --start-time "2024-06-15 00:00" --ndays 1

# Specify time range with --stop-time instead of --ndays
heat1d --use-spice --lat 0 --lon 0 \
       --start-time "2024-06-01 00:00" --stop-time "2024-07-01 00:00"

# Arbitrary body (Bennu) with explicit body ID
heat1d --use-spice --body-id 2101955 --lat 0 --lon 0 \
       --start-time "2024-01-01 00:00" --ndays 1 bennu.yaml
```

## 7. Automatic Eclipse Detection

When `--use-spice` is active and the body is a natural satellite
(Moon, Europa, Titan, etc.), `heat1d` automatically detects eclipses
by querying the parent body's angular position relative to the Sun.

The eclipse fraction is computed from the circle-circle overlap
between the Sun's disk and the parent body's disk, using real
ephemeris geometry from JPL Horizons.  Both queries use the same
observer lat/lon, so the eclipse geometry is position-dependent.

### 7.1 Lunar Eclipse of 2025-Mar-14

We query a 12-hour window around the total lunar eclipse of
2025-Mar-14 from the Moon's near side (lon=0, where Earth is overhead).

In [ ]:
from heat1d.horizons import query_parent_body, apply_horizons_eclipses
from heat1d.eclipse import compute_eclipse_fraction, is_satellite, get_parent_body_id

# Eclipse window: 2025-Mar-14, near side (lon=0)
eclipse_start = "2025-03-14 00:00"
eclipse_stop  = "2025-03-14 12:00"
eclipse_step  = "1m"  # 1-minute resolution for fine detail

# Query Sun (with eclipse quantities)
sun_result = query_horizons(
    body_id="301", lon_deg=0.0, lat_deg=0.0,
    start_time=eclipse_start, stop_time=eclipse_stop,
    step_size=eclipse_step, quantities="4,13,20,25",
)

# Query parent body (Earth) angular diameter
parent_result = query_parent_body(
    parent_id="399", satellite_id="301",
    lon_deg=0.0, lat_deg=0.0,
    start_time=eclipse_start, stop_time=eclipse_stop,
    step_size=eclipse_step,
)

# Compute eclipse fraction
frac = compute_eclipse_fraction(
    sun_result["toi_deg"],
    sun_result["sun_ang_diam_arcsec"],
    parent_result["ang_diam_arcsec"],
)

print(f"Samples: {len(frac)}")
print(f"Max eclipse fraction: {frac.max():.3f}")
print(f"Eclipsed samples (frac > 0): {np.sum(frac > 0)}")
print(f"Total eclipse samples (frac == 1): {np.sum(frac >= 0.999)}")

In [ ]:
# Plot eclipse geometry
t_min = np.arange(len(frac))  # minutes since start

fig, axes = plt.subplots(3, 1, figsize=(10, 8), sharex=True)

# T-O-I angle
ax = axes[0]
toi = sun_result["toi_deg"]
ax.plot(t_min / 60.0, toi, 'k-', lw=0.8)
ax.axhline(0, color='red', ls='--', lw=0.5, alpha=0.7)
ax.set_ylabel("T-O-I angle [deg]")
ax.set_title("2025-Mar-14 Lunar Eclipse — Moon near side (lon=0, lat=0)")

# Eclipse fraction
ax = axes[1]
ax.fill_between(t_min / 60.0, frac, color='navy', alpha=0.6)
ax.plot(t_min / 60.0, frac, 'k-', lw=0.5)
ax.set_ylabel("Eclipse fraction")
ax.set_ylim(-0.05, 1.1)

# Solar flux with eclipse
flux_ecl = horizons_to_flux(
    sun_result["solar_elevation_deg"],
    sun_result["observer_range_au"],
    planets.Moon,
)
flux_no_ecl = flux_ecl.copy()
flux_ecl *= (1.0 - frac[:len(flux_ecl)])

ax = axes[2]
ax.plot(t_min / 60.0, flux_no_ecl, 'gold', lw=1, label='Without eclipse', alpha=0.7)
ax.plot(t_min / 60.0, flux_ecl, 'k-', lw=1, label='With eclipse')
ax.fill_between(t_min / 60.0, flux_ecl, alpha=0.15, color='navy')
ax.set_ylabel("Absorbed flux [W/m$^2$]")
ax.set_xlabel("Hours since 2025-03-14 00:00 UTC")
ax.legend(fontsize=9)

fig.tight_layout()
plt.show()

### 7.2 Thermal Model Through an Eclipse

Run the full thermal model over a lunar day that includes the
2025-Mar-14 eclipse.  The `fetch_solar_flux` function detects
eclipses automatically for satellite bodies.

In [ ]:
# Fetch flux for ~1 lunar day around the eclipse (eclipses auto-detected)
ecl_output_s = planets.Moon.day / 480  # ~88 min, ~5 min resolution

flux_with_ecl, dt_ecl, meta_ecl = fetch_solar_flux(
    planet_name="Moon",
    lon_deg=0.0,
    lat_deg=0.0,
    start_time="2025-03-01 00:00",
    stop_time="2025-03-30 00:00",
    output_interval_s=ecl_output_s,
    eclipses=True,
)

ecl_info = meta_ecl["eclipse_info"]
print(f"Eclipse detection: {ecl_info['n_eclipses']} event(s)")
print(f"Max obscuration:   {ecl_info['max_fraction']*100:.1f}%")
print(f"Eclipsed samples:  {ecl_info['total_eclipse_samples']}")

# Also fetch without eclipses for comparison
flux_no_ecl_model, _, _ = fetch_solar_flux(
    planet_name="Moon",
    lon_deg=0.0,
    lat_deg=0.0,
    start_time="2025-03-01 00:00",
    stop_time="2025-03-30 00:00",
    output_interval_s=ecl_output_s,
    eclipses=False,
)

# Run both models
config_ecl = heat1d.Configurator(solver="implicit", output_interval=dt_ecl)
ndays_ecl = len(flux_with_ecl) * dt_ecl / planets.Moon.day

model_ecl = heat1d.Model(
    planet=planets.Moon, lat=0.0, ndays=ndays_ecl, config=config_ecl,
    flux_series=flux_with_ecl, flux_dt=dt_ecl,
)
model_ecl.run()

model_no_ecl = heat1d.Model(
    planet=planets.Moon, lat=0.0, ndays=ndays_ecl, config=config_ecl,
    flux_series=flux_no_ecl_model, flux_dt=dt_ecl,
)
model_no_ecl.run()

print(f"\nWith eclipse:    T_min = {model_ecl.T[:, 0].min():.1f} K")
print(f"Without eclipse: T_min = {model_no_ecl.T[:, 0].min():.1f} K")

In [ ]:
# Plot surface temperature through the eclipse
t_model_hr_ecl = model_ecl.lt / 24.0 * planets.Moon.day / 3600.0
t_model_hr_ne = model_no_ecl.lt / 24.0 * planets.Moon.day / 3600.0

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

# Flux
t_flux_hr = np.arange(len(flux_with_ecl)) * dt_ecl / 3600.0
ax1.plot(t_flux_hr, flux_no_ecl_model, color='gold', lw=0.8, alpha=0.7,
         label='No eclipse')
ax1.plot(t_flux_hr, flux_with_ecl, 'k-', lw=0.8, label='With eclipse')
ax1.set_ylabel('Absorbed flux [W/m$^2$]')
ax1.legend(fontsize=9)
ax1.set_title('Thermal model through 2025-Mar-14 lunar eclipse (lon=0)')

# Temperature
ax2.plot(t_model_hr_ne, model_no_ecl.T[:, 0], color='gold', lw=1, alpha=0.7,
         label='No eclipse')
ax2.plot(t_model_hr_ecl, model_ecl.T[:, 0], 'k-', lw=1, label='With eclipse')
ax2.set_ylabel('Surface temperature [K]')
ax2.set_xlabel('Time since start [Earth hours]')
ax2.legend(fontsize=9)

fig.tight_layout()
plt.show()

# Temperature dip
T_diff = model_no_ecl.T[:, 0] - model_ecl.T[:, 0]
print(f"Max temperature reduction from eclipse: {T_diff.max():.1f} K")

### 7.3 Position Dependence

During a total lunar eclipse, Earth's umbral shadow cone
(~9,000 km diameter at lunar distance) is much larger than the
Moon (~3,474 km), so the entire Moon is in shadow.  However,
the eclipse geometry still varies slightly with observer
position due to parallax.

We compare the eclipse fraction from three longitudes on the
equator to show how the T-O-I angle changes with position.

In [ ]:
# Query eclipse geometry from three different longitudes
lons = [0, 90, 180]
colors = ['navy', 'teal', 'coral']
labels = ['lon=0 (near side)', 'lon=90 (limb)', 'lon=180 (far side)']

fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

for lon, color, label in zip(lons, colors, labels):
    sun_r = query_horizons(
        body_id="301", lon_deg=lon, lat_deg=0.0,
        start_time=eclipse_start, stop_time=eclipse_stop,
        step_size="2m", quantities="4,13,20,25",
    )
    par_r = query_parent_body(
        parent_id="399", satellite_id="301",
        lon_deg=lon, lat_deg=0.0,
        start_time=eclipse_start, stop_time=eclipse_stop,
        step_size="2m",
    )
    f = compute_eclipse_fraction(
        sun_r["toi_deg"], sun_r["sun_ang_diam_arcsec"],
        par_r["ang_diam_arcsec"],
    )
    t_h = np.arange(len(f)) * 2.0 / 60.0  # 2-min steps → hours

    axes[0].plot(t_h, sun_r["toi_deg"], color=color, lw=1, label=label)
    axes[1].plot(t_h, f, color=color, lw=1, label=label)

    print(f"{label}: max eclipse fraction = {f.max():.3f}, "
          f"eclipsed samples = {np.sum(f > 0)}")

axes[0].axhline(0, color='red', ls='--', lw=0.5, alpha=0.7)
axes[0].set_ylabel("T-O-I angle [deg]")
axes[0].set_title("Eclipse geometry vs. observer longitude")
axes[0].legend(fontsize=9)

axes[1].set_ylabel("Eclipse fraction")
axes[1].set_xlabel("Hours since 2025-03-14 00:00 UTC")
axes[1].set_ylim(-0.05, 1.1)
axes[1].legend(fontsize=9)

fig.tight_layout()
plt.show()

### Eclipse CLI Usage

Eclipse detection is automatic when `--use-spice` is active and the
body is a natural satellite.  Use `--no-eclipses` to disable it.

```bash
# Moon near side — eclipse auto-detected
heat1d --use-spice --lat 0 --lon 0 \
       --start-time "2025-03-01 00:00" --ndays 1

# Disable eclipse detection
heat1d --use-spice --lat 0 --lon 0 --no-eclipses \
       --start-time "2025-03-01 00:00" --ndays 1

# Manual parent body override (e.g., for asteroid moons)
heat1d --use-spice --body-id 501 --lat 0 --lon 0 \
       --parent-body-id 599 \
       --start-time "2025-03-01 00:00" --ndays 1
```

Or via YAML configuration:
```yaml
horizons:
  enabled: true
  eclipses:
    enabled: true
    # parent_body_id: "399"  # auto-detected for known satellites
```